<a href="https://colab.research.google.com/github/Aswin-Cheerngodan/Text-Generation/blob/main/Text_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the dependencies

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [13]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,SimpleRNN,Embedding,Dropout,BatchNormalization,Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.utils as ku

Data loading and preprocessing

In [14]:
data = pd.read_csv('/content/drive/MyDrive/Keras,Pytorch,Tensorflow/Text Generator/ArticlesApril2017.csv')
data.head()

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,58def1347c459f24986d7c80,716,By STEPHEN HILTNER and SUSAN LEHMAN,article,Finding an Expansive View of a Forgotten Peop...,"['Photography', 'New York Times', 'Niger', 'Fe...",3,Insider,2,2017-04-01 00:15:41,Unknown,One of the largest photo displays in Times his...,The New York Times,News,https://www.nytimes.com/2017/03/31/insider/nig...
1,NaN,58def3237c459f24986d7c84,823,By GAIL COLLINS,article,"And Now, the Dreaded Trump Curse","['United States Politics and Government', 'Tru...",3,OpEd,23,2017-04-01 00:23:58,Unknown,Meet the gang from under the bus.,The New York Times,Op-Ed,https://www.nytimes.com/2017/03/31/opinion/and...
2,NaN,58def9f57c459f24986d7c90,575,By THE EDITORIAL BOARD,article,Venezuela’s Descent Into Dictatorship,"['Venezuela', 'Politics and Government', 'Madu...",3,Editorial,22,2017-04-01 00:53:06,Unknown,A court ruling annulling the legislature’s aut...,The New York Times,Editorial,https://www.nytimes.com/2017/03/31/opinion/ven...
3,NaN,58defd317c459f24986d7c95,1374,By MICHAEL POWELL,article,Stain Permeates Basketball Blue Blood,"['Basketball (College)', 'University of North ...",3,Sports,1,2017-04-01 01:06:52,College Basketball,"For two decades, until 2013, North Carolina en...",The New York Times,News,https://www.nytimes.com/2017/03/31/sports/ncaa...
4,NaN,58df09b77c459f24986d7ca7,708,By DEB AMLEN,article,Taking Things for Granted,['Crossword Puzzles'],3,Games,0,2017-04-01 02:00:14,Unknown,In which Howard Barkin and Will Shortz teach u...,The New York Times,News,https://www.nytimes.com/2017/03/31/crosswords/...


In [15]:
all_headline = list(data['headline'].values)

Data Cleaning

In [16]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 9.4 MB/s eta 0:00:00


In [17]:
import re
import contractions

In [18]:
def clean_txt(text):
  text = contractions.fix(text)
  text = re.sub(r'[^a-zA-Z\s]','',text)
  text = text.lower()
  return text

In [19]:
headlines = [clean_txt(text) for text in all_headline]

In [20]:
tokenizer = Tokenizer()

def get_seq_of_toks(corpus):
  tokenizer.fit_on_texts(corpus)
  total_words = len(tokenizer.word_index)+1

  input_sequences = []
  for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)-1):
      n_gram_seq = token_list[:i+1]
      input_sequences.append(n_gram_seq)

  return input_sequences,total_words

In [21]:
input_seq,total_words = get_seq_of_toks(headlines)

In [22]:
def generate_pad_seqs(input_seqs):
  max_len = max([len(x) for x in input_seqs])
  input_seqs = np.array(pad_sequences(input_seqs,maxlen=max_len,padding='pre'))
  predictors,label = input_seqs[:,:-1],input_seqs[:,-1]
  label = ku.to_categorical(label,num_classes=total_words)
  return predictors,label,max_len


In [23]:
predictors,label,max_seq_len = generate_pad_seqs(input_seq)

In [24]:
print(total_words)

2366


In [32]:
model = Sequential([
    Input(shape=(max_seq_len-1,)),
    Embedding(total_words,10),
    SimpleRNN(200,return_sequences=True),
    Dropout(0.1),
    SimpleRNN(200),
    Dropout(0.1),
    Dense(total_words,activation='softmax')
])

In [33]:
model.compile(loss='categorical_crossentropy',optimizer='adam')
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 17, 10)              │          23,660 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_4 (SimpleRNN)             │ (None, 17, 200)             │          42,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 17, 200)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_5 (SimpleRNN)             │ (None, 200)                 │          80,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2366)                │         475,566 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 621,626 (2.37 MB)

 Trainable params: 621,626 (2.37 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
model.fit(predictors,label,epochs=100)

Epoch 1/100
124/124 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - loss: 7.2146
Epoch 2/100
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.5948
Epoch 3/100
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 6.6611
Epoch 4/100
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 7.0296
Epoch 5/100
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 6.6148
Epoch 6/100
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 6.6681
Epoch 7/100
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.6876
Epoch 8/100
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.7285
Epoch 9/100
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.5730
Epoch 10/100
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.6552
Epoch 11/100
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.6499
Epoch 12/100
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.6810
Epoch 13/100
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.6763
Epoch 14/100
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.7145
Epoch 15/100
124/124 ━━━━━━━━━━━━━━━━━━━━ 

In [49]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],
        maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)

        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index ==np.argmax(predicted):
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [51]:
print (generate_text("india and pakistan", 10, model, max_seq_len))
print (generate_text("president trump", 3, model, max_seq_len))
print (generate_text("united states", 4, model, max_seq_len))
print (generate_text("donald trump", 2, model, max_seq_len))
print (generate_text("new york", 3, model, max_seq_len))
print (generate_text("science and technology", 5, model, max_seq_len))

India And Pakistan It Is Not Offlimits To Artists For A Ugly They
President Trump For Pot Farms
United States Still From Trump It
Donald Trump Plays By
New York Today An Ella
Science And Technology It Is Not Offlimits To
